Show, analyze and visualize the LSO results.

In [ ]:
import numpy as np

data_file = '/pfs/work9/workspace/scratch/ma_mgraevin-optdif/results/lso_dngo_03/bo/iter_0/data.npz'

with np.load(data_file, allow_pickle=True) as npz:
    print(npz.files)
    X_train = npz['X_train'].astype(np.float32)
    y_train = npz['y_train'].astype(np.float32)

In [ ]:
# Display tensor (3, 265, 256) as image
import matplotlib.pyplot as plt
import numpy as np

def show_tensor(tensor):
    # Convert to numpy array
    tensor = (tensor + 1) / 2
    img = tensor.permute(1, 2, 0).numpy()
    # Clip values to [0, 1]
    img = np.clip(img, 0, 1)
    # Display the image
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
import torch

tensor_0 = torch.load('/pfs/work9/workspace/scratch/ma_mgraevin-optdif/results/debug_10/data/samples/iter_0/img_tensor/tensor_4.pt', weights_only=False)

show_tensor(tensor_0)

In [ ]:
tensor_0.min(), tensor_0.max(), tensor_0.mean(), tensor_0.std()

In [ ]:
# npz file to table
import pandas as pd
import numpy as np
results = np.load("/pfs/work9/workspace/scratch/ma_mgraevin-optdif/results/lso_gp_01/results.npz", allow_pickle=True)

print(f"Parameters: {results['params']}")

# Convert the npz file to a pandas DataFrame
df = pd.DataFrame({
    #'opt_points': list(results['opt_points']),
    'opt_point_properties': list(results['opt_point_properties']),
    #'opt_latent_points': list(results['opt_latent_points']),
    #'opt_sd_latent_points': list(results['opt_sd_latent_points']),
    'opt_model_version': list(results['opt_model_version']),
})

df

In [ ]:
df_grouped = df.groupby('opt_model_version')['opt_point_properties'].apply(list).apply(lambda x: x[:5]).apply(pd.Series)
df_grouped.columns = [f'tensor_{i}' for i in range(5)]
df_grouped

In [ ]:
df_grouped_mean = df_grouped.mean(axis=1)
df_grouped_mean = df_grouped_mean.reset_index()
df_grouped_mean.columns = ['opt_model_version', 'mean']
df_grouped_mean

In [ ]:
# group and extract mean, min, and max over the first 5 tensors
df_grouped_stats = df_grouped.agg(['mean', 'min', 'max'], axis=1).reset_index()
df_grouped_stats

In [ ]:
# visualize the mean values
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_grouped_mean['opt_model_version'], df_grouped_stats['mean'], marker='o', label='Mean')
ax.fill_between(df_grouped_mean['opt_model_version'], df_grouped_stats['min'], df_grouped_stats['max'], alpha=0.2, label='Min-Max Range')
ax.legend()
ax.set_xlabel('Iteration')
ax.set_ylabel('Values')
ax.set_title('Mean Values of the five samples per Iteration')
ax.set_ylim(0, 5)
ax.axhline(y=1, color='red', linestyle='--')
plt.xticks(ticks=range(len(df_grouped_mean['opt_model_version'])), labels=df_grouped_mean['opt_model_version'], rotation=45)
plt.tight_layout()
plt.show()